In [1]:
import numpy as np 
import pandas as pd 
import requests
from tqdm import tqdm 
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
link = 'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true'    # this link belongs to electronics/ computer based products
driver.get(link)
soup = BeautifulSoup(driver.page_source,'html.parser')

[WDM] - Downloading: 100%|██████████████████| 6.83M/6.83M [00:21<00:00, 333kB/s]
/tmp/ipykernel_4734/3157396036.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Scraping the Dataset of computer and accessories (scraping one dataset)

In [192]:
data = []

link = 'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true'
for pages in tqdm(range(2,401)):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    for info in soup.find_all('div',{'class' : 'a-section a-spacing-base'}):
        # name of the product 
        name = info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-title-instructions-style').find('h2').text
        # rating obtained by that product 
        try:
            if ' ' in info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text:
                stars = float(info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text[:3])
            else:
                stars= 'not available'
        except:
            stars = 'not available'

        # price of the product 
        try:
            sell_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price').find('span').text[1:].replace(',',''))
        except:
            sell_price = 0
        try:
            mark_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price a-text-price').find('span',class_ = 'a-offscreen').text[1:].replace(',',''))
        except:
            mark_price = 0


        # delivery date 
        try:
            delivered_on = info.find('div',class_ =  'a-row a-size-base a-color-secondary s-align-children-center').find('span',class_ = 'a-color-base a-text-bold').text
        except:
            delivered_on = 'Data not available'

        data.append([name,stars,sell_price,mark_price,delivered_on])
        
    link = f'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true&page={pages}&qid=1679465284&ref=sr_pg_{pages}'

100%|█████████████████████████████████████████| 399/399 [37:22<00:00,  5.62s/it]


In [193]:
product = pd.DataFrame(data,columns =['name','stars','sell_price','mark_price','delivery_date'])
product.to_csv('product.csv',index = False)

In [2]:
df = pd.read_csv('product.csv')
df.head()

,name,stars,sell_price,mark_price,delivery_date
0,SHINING SUN VISION Smart LED Vision Chart 19 i...,3.5,19499,30000,"Wednesday, 29 March"
1,"Regatech Compatible for Gateway NV53A46U, NV53...",not available,799,2499,"Tuesday, 28 March"
2,"Regatech Compatible for Hp G62-A70SQ, G62-A70S...",not available,699,2499,"Tuesday, 28 March"
3,boAt Airdopes 141 Bluetooth Truly Wireless in ...,3.9,0,0,"Tomorrow, 23 March"
4,boAt BassHeads 100 in-Ear Wired Headphones wit...,4.1,379,999,"Today, 22 March"


### Getting the information about the product.csv file (Performing exploration on the given data)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13075 entries, 0 to 13074
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           13075 non-null  object
 1   stars          13075 non-null  object
 2   sell_price     13075 non-null  int64 
 3   mark_price     13075 non-null  int64 
 4   delivery_date  13075 non-null  object
dtypes: int64(2), object(3)
memory usage: 510.9+ KB


In [7]:
df.describe()

,sell_price,mark_price
count,13075.000000,13075.000000
mean,4259.728719,7336.086883
std,10080.744019,15218.477823
min,0.000000,0.000000
25%,219.000000,299.000000
50%,900.000000,1999.000000
75%,3999.000000,9916.500000
max,199900.000000,400000.000000


## Creating the dictionary of the links 

In [3]:
dct = {
    'mobilecover'  : 's?rh=n%3A1389409031&fs=true',
    'laptop'       : 's?rh=n%3A1375424031&fs=true',
    'computer'     : 's?rh=n%3A976392031&fs=true',
    'cricket'      : 's?rh=n%3A3403628031&fs=true',
    'badminton'    : 's?rh=n%3A3403619031&fs=true',
    'motorbike'    : 's?rh=n%3A5257478031&fs=true',
}


### Scraping the data of all the pages along with product

In [25]:
scrape = input('Enter the name of thing that you are looking of :').lower()

if scrape in dct.keys():
    data = []
    link = 'https://www.amazon.in/' + dct[scrape]
    for pages in tqdm(range(2,401)):
        driver.get(link)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        for info in soup.find_all('div',{'class' : 'a-section a-spacing-base'}):
            # name of the product 
            name = info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-title-instructions-style').find('h2').text
            # rating obtained by that product 
            try:
                if ' ' in info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text:
                    stars = float(info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text[:3])
                else:
                    stars= 'not available'
            except:
                stars = 'not available'

            # price of the product 
            try:
                sell_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price').find('span').text[1:].replace(',',''))
            except:
                sell_price = 0
            try:
                mark_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price a-text-price').find('span',class_ = 'a-offscreen').text[1:].replace(',',''))
            except:
                mark_price = 0

            # delivery date 
            try:
                delivered_on = info.find('div',class_ =  'a-row a-size-base a-color-secondary s-align-children-center').find('span',class_ = 'a-color-base a-text-bold').text
            except:
                delivered_on = 'Data not available'

            data.append([name,stars,sell_price,mark_price,delivered_on])

        link = 'https://www.amazon.in/' + dct[scrape] + f'&page={pages}&ref=sr_pg_{pages}'
else:
    print('Not present')


product = pd.DataFrame(data,columns =['name','stars','sell_price','mark_price','delivery_date'])
product.to_csv(scrape + '.csv',index = False)

Enter the name of thing that you are looking of :womenwears


100%|█████████████████████████████████████████| 399/399 [40:21<00:00,  6.07s/it]


### Setting the default column for all the products

In [26]:
df1 = pd.read_csv('product.csv')
df1['product_type'] = 'Computers and Peripherals'

df2 =  pd.read_csv('laptop.csv')
df2['product_type'] = 'Laptops'

df3 =  pd.read_csv('mobliecovers.csv')
df3['product_type'] = 'Moblie Covers'

df4 =  pd.read_csv('cricket.csv')
df4['product_type'] = 'Cricket Equipments and Essentials'

df5 = pd.read_csv('badminton.csv')
df5['product_type'] = 'Badminton Accessories and Essentials'

df6 = pd.read_csv('motorbike.csv')
df6['product_type'] = 'Motorbike Accessories and Essentials'


In [71]:
stores = pd.concat([df1,df2,df3,df4,df5,df6])
stores.to_csv('products.csv',index = False)

In [4]:
df = pd.read_csv('products.csv')

In [5]:
# pre processing the stars column

In [6]:
ls = []
for i in df['stars']:
    if i == 'not available':
        ls.append(0.0)
    else:
        ls.append(i)
        
df['stars'] = ls